<a href="https://colab.research.google.com/github/ShuvoProgram/AmazonProductScrape/blob/main/AmazonProduct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandas

In [3]:
pip install bs4

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=mobile&_sacat=0'

**Create a dictionary**

In [6]:
d = {'key' : 'value'}
print(d)

{'key': 'value'}


**Update the dictionary**

In [7]:
d['new key'] = 'new value'
print(d)

{'key': 'value', 'new key': 'new value'}


In [ ]:
value_data = {}
page_no = 0

while True:
  data = requests.get(url)
  soup = BeautifulSoup(data.content, 'html.parser')
  product = soup.find_all('li', {'class': 's-item__pl-on-bottom'})

  for products in product:
    title = products.find('h3', {'class': 's-item__title'}).text
    price = soup.find('span', {'class': 's-item__price'}).text
    img = products.find('img', {'class': 's-item__image-img'})
    # imgs = img.get('src') if link else 'N/A'
    link = products.find('a', {'class': 's-item__link'})
    links = link.get('href') if link else 'N/A'

    details = requests.get(links)
    details_data = details.text
    details_soup = BeautifulSoup(details_data, 'html.parser')
    # description = details_soup.find('div', {'class': 'tab-content-m'})
    eBay_item_number = details_soup.find('div', {'id': 'descItemNumber'})
    eBay_item_number_tag = eBay_item_number.text if eBay_item_number else 'N/A'

    page_no+=1
    value_data[page_no] = [title, price, img, links, eBay_item_number_tag]

    # print('product_title: ', title, '\nproduct_price: ', price, '\nproduct_image: ', imgs, '\nlink: ', link, '\neBay_item_number: ', eBay_item_number)

url_tag = soup.find('a', {'class': 'pagination__next icon-link'})
if url_tag.get('href'):
  url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=mobile&_sacat=0' + url_tag.get('href')
  print(url)
else:
  break
print('Total Product: ', page_no)

value_data_df = pd.DataFrame.from_dict(value_data, orient= 'index', columns= ['product_title', 'product_price', 'product_image', 'link', 'eBay_item_number'])

value_data_df.head()

value_data_df.to_csv('mobile.csv')
